In [ ]:
import polars as pl


df_2020 = pl.read_parquet("/Users/vsevolod/Desktop/divvy-bikes-analysis/data/processed/2020/rides_2020_optimized_filled.parquet")
df_2021 = pl.read_parquet("/Users/vsevolod/Desktop/divvy-bikes-analysis/data/processed/2021/rides_2021_optimized_filled.parquet")
df_2022 = pl.read_parquet("/Users/vsevolod/Desktop/divvy-bikes-analysis/data/processed/2022/rides_2022_optimized_filled.parquet")
df_2023 = pl.read_parquet("/Users/vsevolod/Desktop/divvy-bikes-analysis/data/processed/2023/rides_2023_optimized_filled.parquet")
df_2024 = pl.read_parquet("/Users/vsevolod/Desktop/divvy-bikes-analysis/data/processed/2024/rides_2024_optimized_filled.parquet")
df_2025 = pl.read_parquet("/Users/vsevolod/Desktop/divvy-bikes-analysis/data/processed/2025/rides_2025_optimized_filled.parquet")

In [1]:
import polars as pl
import os

# Список путей к файлам
file_paths = [
    "/Users/vsevolod/Desktop/divvy-bikes-analysis/data/processed/2020/rides_2020_optimized_filled.parquet",
    "/Users/vsevolod/Desktop/divvy-bikes-analysis/data/processed/2021/rides_2021_optimized_filled.parquet",
    "/Users/vsevolod/Desktop/divvy-bikes-analysis/data/processed/2022/rides_2022_optimized_filled.parquet",
    "/Users/vsevolod/Desktop/divvy-bikes-analysis/data/processed/2023/rides_2023_optimized_filled.parquet",
    "/Users/vsevolod/Desktop/divvy-bikes-analysis/data/processed/2024/rides_2024_optimized_filled.parquet",
    "/Users/vsevolod/Desktop/divvy-bikes-analysis/data/processed/2025/rides_2025_optimized_filled.parquet"
]

# Конвертируем каждый файл в CSV отдельно
for i, path in enumerate(file_paths):
    if os.path.exists(path):  # Проверяем существование файла
        year = 2020 + i
        df = pl.read_parquet(path)
        csv_path = f"/Users/vsevolod/Desktop/divvy-bikes-analysis/data/processed/rides_{year}.csv"
        df.write_csv(csv_path)
        print(f"Сохранен: {csv_path} (строк: {df.height:,})")
    else:
        print(f"Файл не найден: {path}")

# Создаем один объединенный файл
print("\nСоздание объединенного файла...")
all_dfs = []

for path in file_paths:
    if os.path.exists(path):
        df = pl.read_parquet(path)
        all_dfs.append(df)

if all_dfs:
    combined_df = pl.concat(all_dfs)
    combined_csv_path = "/Users/vsevolod/Desktop/divvy-bikes-analysis/data/processed/all_rides_2020_2025.csv"
    combined_df.write_csv(combined_csv_path)
    print(f"Объединенный файл сохранен: {combined_csv_path}")
    print(f"Итого строк: {combined_df.height:,}")
    print(f"Колонки: {combined_df.columns}")
else:
    print("Нет данных для объединения")

Сохранен: /Users/vsevolod/Desktop/divvy-bikes-analysis/data/processed/rides_2020.csv (строк: 3,468,881)
Сохранен: /Users/vsevolod/Desktop/divvy-bikes-analysis/data/processed/rides_2021.csv (строк: 4,800,858)
Сохранен: /Users/vsevolod/Desktop/divvy-bikes-analysis/data/processed/rides_2022.csv (строк: 5,437,102)
Сохранен: /Users/vsevolod/Desktop/divvy-bikes-analysis/data/processed/rides_2023.csv (строк: 5,563,046)
Сохранен: /Users/vsevolod/Desktop/divvy-bikes-analysis/data/processed/rides_2024.csv (строк: 5,583,576)
Сохранен: /Users/vsevolod/Desktop/divvy-bikes-analysis/data/processed/rides_2025.csv (строк: 4,916,696)

Создание объединенного файла...


ShapeError: unable to vstack, column names don't match: "same_start_end" and "start_hour"

In [2]:
import polars as pl
import os

# ============================================================================
# КОНФИГУРАЦИЯ
# ============================================================================

DATA_PATHS = {
    2020: "/Users/vsevolod/Desktop/divvy-bikes-analysis/data/processed/2020/rides_2020_optimized_filled.parquet",
    2021: "/Users/vsevolod/Desktop/divvy-bikes-analysis/data/processed/2021/rides_2021_optimized_filled.parquet",
    2022: "/Users/vsevolod/Desktop/divvy-bikes-analysis/data/processed/2022/rides_2022_optimized_filled.parquet",
    2023: "/Users/vsevolod/Desktop/divvy-bikes-analysis/data/processed/2023/rides_2023_optimized_filled.parquet",
    2024: "/Users/vsevolod/Desktop/divvy-bikes-analysis/data/processed/2024/rides_2024_optimized_filled.parquet",
    2025: "/Users/vsevolod/Desktop/divvy-bikes-analysis/data/processed/2025/rides_2025_optimized_filled.parquet"
}

SAVE_DIR = "/Users/vsevolod/Desktop/divvy-bikes-analysis/visualizations"
CSV_SAVE_DIR = "/Users/vsevolod/Desktop/divvy-bikes-analysis/data/processed/csv"
os.makedirs(SAVE_DIR, exist_ok=True)
os.makedirs(CSV_SAVE_DIR, exist_ok=True)

# ============================================================================
# ОБЩИЕ ФУНКЦИИ ДЛЯ ЗАГРУЗКИ ДАННЫХ
# ============================================================================

def load_year_data(year, columns=None):
    """Загружает данные за конкретный год с ленивой обработкой"""
    if year not in DATA_PATHS:
        return None
    
    path = DATA_PATHS[year]
    if not os.path.exists(path):
        return None
    
    # Базовые колонки, которые всегда нужны
    base_columns = ["started_at", "ended_at", "member_casual", "rideable_type"]
    
    if columns:
        base_columns = [col for col in columns if col in base_columns + ["start_station_name", "end_station_name", "start_lat", "start_lng"]]
    
    # Ленивая загрузка
    lf = pl.scan_parquet(path).select(base_columns)
    
    # Добавляем вычисляемые колонки
    lf = lf.with_columns([
        pl.lit(year).alias("year"),
        pl.col("started_at").dt.month().alias("month"),
        pl.col("started_at").dt.hour().alias("hour"),
        pl.col("started_at").dt.weekday().alias("weekday"),
        ((pl.col("ended_at") - pl.col("started_at")).dt.total_seconds()).alias("duration_sec")
    ])
    
    # Фильтруем аномалии
    lf = lf.filter(
        (pl.col("duration_sec") > 60) & 
        (pl.col("duration_sec") < 24 * 3600)
    )
    
    return lf

def collect_safely(lazy_frame):
    """Безопасный сбор данных с обработкой ошибок"""
    try:
        return lazy_frame.collect(streaming=True)
    except Exception as e:
        print(f"Ошибка при сборе данных: {e}")
        return None

# ============================================================================
# СОХРАНЕНИЕ В CSV
# ============================================================================

def save_individual_csvs():
    """Сохраняет каждый годовой датасет в отдельный CSV файл"""
    print("Сохранение годовых данных в CSV...")
    
    for year in DATA_PATHS:
        print(f"\nОбработка {year} года...")
        
        # Загружаем данные
        lazy_data = load_year_data(year)
        if lazy_data is None:
            print(f"  Пропущено: файл не найден или ошибка загрузки")
            continue
        
        # Собираем данные
        df = collect_safely(lazy_data)
        if df is None or df.height == 0:
            print(f"  Пропущено: нет данных после фильтрации")
            continue
        
        # Сохраняем в CSV
        csv_path = os.path.join(CSV_SAVE_DIR, f"rides_{year}.csv")
        df.write_csv(csv_path)
        print(f"  ✓ Сохранено: {csv_path}")
        print(f"    Количество строк: {df.height:,}")
        print(f"    Колонки: {', '.join(df.columns[:5])}...")

def save_combined_csv():
    """Объединяет все данные и сохраняет в один CSV файл"""
    print("\n" + "="*60)
    print("Создание объединенного файла...")
    print("="*60)
    
    all_lazy_frames = []
    
    # Собираем все ленивые фреймы
    for year in DATA_PATHS:
        print(f"Загрузка {year} года...")
        lazy_data = load_year_data(year)
        if lazy_data is not None:
            all_lazy_frames.append(lazy_data)
    
    if not all_lazy_frames:
        print("Нет данных для объединения")
        return
    
    # Объединяем ленивые фреймы
    print("Объединение данных...")
    combined_lf = pl.concat(all_lazy_frames, how="diagonal")
    
    # Собираем и сохраняем
    combined_df = collect_safely(combined_lf)
    if combined_df is None or combined_df.height == 0:
        print("Ошибка: не удалось объединить данные")
        return
    
    # Сохраняем объединенный файл
    combined_csv_path = os.path.join(CSV_SAVE_DIR, "all_rides_2020_2025.csv")
    combined_df.write_csv(combined_csv_path)
    
    print(f"\n✓ Объединенный файл сохранен: {combined_csv_path}")
    print(f"  Итого строк: {combined_df.height:,}")
    print(f"  Количество колонок: {len(combined_df.columns)}")
    print(f"  Диапазон дат: {combined_df['started_at'].min()} - {combined_df['started_at'].max()}")
    
    # Статистика по годам
    print("\n  Распределение по годам:")
    year_counts = combined_df.group_by("year").agg(pl.count().alias("count"))
    for row in year_counts.iter_rows():
        print(f"    {row[0]}: {row[1]:,} поездок")

# ============================================================================
# ОСНОВНОЙ БЛОК
# ============================================================================

if __name__ == "__main__":
    print("Начало обработки данных...")
    print(f"Папка для CSV: {CSV_SAVE_DIR}")
    
    # Сохраняем отдельные CSV файлы
    save_individual_csvs()
    
    # Сохраняем объединенный CSV файл
    save_combined_csv()
    
    print("\n" + "="*60)
    print("Обработка завершена!")
    print("="*60)

Начало обработки данных...
Папка для CSV: /Users/vsevolod/Desktop/divvy-bikes-analysis/data/processed/csv
Сохранение годовых данных в CSV...

Обработка 2020 года...


/var/folders/m_/0vg5klms5wqfl8z1jkpk95tr0000gn/T/ipykernel_10330/2991596543.py:64: DeprecationWarning: the `streaming` parameter was deprecated in 1.25.0; use `engine` instead.
  return lazy_frame.collect(streaming=True)


  ✓ Сохранено: /Users/vsevolod/Desktop/divvy-bikes-analysis/data/processed/csv/rides_2020.csv
    Количество строк: 3,468,498
    Колонки: started_at, ended_at, member_casual, rideable_type, year...

Обработка 2021 года...
  ✓ Сохранено: /Users/vsevolod/Desktop/divvy-bikes-analysis/data/processed/csv/rides_2021.csv
    Количество строк: 4,800,858
    Колонки: started_at, ended_at, member_casual, rideable_type, year...

Обработка 2022 года...
  ✓ Сохранено: /Users/vsevolod/Desktop/divvy-bikes-analysis/data/processed/csv/rides_2022.csv
    Количество строк: 5,437,102
    Колонки: started_at, ended_at, member_casual, rideable_type, year...

Обработка 2023 года...
  ✓ Сохранено: /Users/vsevolod/Desktop/divvy-bikes-analysis/data/processed/csv/rides_2023.csv
    Количество строк: 5,561,592
    Колонки: started_at, ended_at, member_casual, rideable_type, year...

Обработка 2024 года...
  ✓ Сохранено: /Users/vsevolod/Desktop/divvy-bikes-analysis/data/processed/csv/rides_2024.csv
    Количество

/var/folders/m_/0vg5klms5wqfl8z1jkpk95tr0000gn/T/ipykernel_10330/2991596543.py:139: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  year_counts = combined_df.group_by("year").agg(pl.count().alias("count"))


    2020: 3,468,498 поездок
    2021: 4,800,858 поездок
    2022: 5,437,102 поездок
    2023: 5,561,592 поездок
    2024: 5,582,518 поездок
    2025: 4,915,694 поездок

Обработка завершена!
